In [1]:
from bs4 import BeautifulSoup
import time
import re
from tqdm.auto import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [2]:
import pandas as pd
df = pd.read_csv('Brunch 키워드.csv'); df

,키워드,서브키워드,토픽,토픽_URL
0,지구한바퀴세계여행,해외여행,자유여행,/keyword/자유여행
1,지구한바퀴세계여행,해외여행,몽골여행,/keyword/몽골여행
2,지구한바퀴세계여행,해외여행,여권,/keyword/여권
3,지구한바퀴세계여행,해외여행,패키지여행,/keyword/패키지여행
4,지구한바퀴세계여행,해외여행,여행지,/keyword/여행지
...,...,...,...,...
6096,감성에세이,산문집,기록,/keyword/기록
6097,감성에세이,산문집,우울증,/keyword/우울증
6098,감성에세이,산문집,사진,/keyword/사진
6099,감성에세이,산문집,글,/keyword/글


In [3]:
topic_list = list(df['토픽_URL'].drop_duplicates())

In [4]:
s = Service('/Users/macair/Documents/chromedriver')
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=s) # options = options
driver.maximize_window()
time.sleep(2)

In [5]:
test_url = df['토픽_URL'][0]
base_url = 'https://brunch.co.kr'
driver.get(base_url + test_url)

In [6]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [14]:
driver.quit()

In [10]:
article_body = soup.find('ul', 'list_article list_common')

In [11]:
article_list = article_body.find_all('li')

In [16]:
test_article = article_list[0]

In [17]:
def get_content_info(soup):
    article_url = soup.find('a').get('href')
    title = soup.find('strong', 'tit_subject').text
    sub_title = soup.find('em', 'tit_sub').text
    author = soup.find_all('span', 'name_txt')[-1].text
    content = soup.find('span', 'article_content').text
    time = soup.find('span', 'publish_time').text
    
    data = [article_url, title, sub_title, author, content, time]
    
    return data

In [18]:
get_content_info(test_article)

['/@@fiTV/20',
 '로키의 꽃 "아이스필드 파크웨이"',
 '캐나다 기행문 7',
 '서진이네',
 'DAY6 -\xa0재스퍼\xa0to\xa0밴프 DAY6는\xa0재스퍼에서 밴프로 이동하는 날이다. 재스퍼에서 밴프까지는 290km나 나오지만, 이미 800km 가까이 드라이브를 했던 터라 290km는 ‘별 거 없네’ 수준이었다. 하지만 이 두 지역의 사이 길에 봐야할 관광지가 몰려있었기 때문에 부지런히 출발해야했다. 재스퍼에서 밴프로 가는 길은 ‘Icefield Parkway’',
 'Aug 10. 2023']

In [19]:
def get_content_url(drive, topic):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    li_list = soup.find('ul', 'list_article list_common').find_all('li')
    
    return list(map(lambda x: get_content_info(x) + [topic], li_list))

## 스크롤 다운 | 함수 | 자동화

In [20]:
s = Service('/Users/macair/Documents/chromedriver')
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=s) # options = options
driver.maximize_window()
time.sleep(2)

In [21]:
test_url = df['토픽_URL'][0]
base_url = 'https://brunch.co.kr'
driver.get(base_url + test_url)

In [25]:
from selenium.webdriver.common.keys import Keys

# 스크롤 다운 방법
body = driver.find_element('css selector', 'body')
body.send_keys(Keys.END)

In [26]:
# 스크롤 5번 내리기
for rep in range(5):
    body.send_keys(Keys.END)
    time.sleep(1)

In [27]:
# 스크롤을 페이지 끝까지 내리는 코드

start = time.time()
last_height = driver.execute_script("return document.body.scrollHeight")
n = 0

while n < 5:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)  # 스크롤이 끝까지 내려가기를 기다리는 시간
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    n += 1
    
end = time.time()
print(n, '번', end - start, '초 소요')

5 번 5.11748194694519 초 소요


In [30]:
content_list = []
base_url = 'https://brunch.co.kr'

start_index = 0
start_topic = topic_list[start_index].split('/')[-1]

end_index = 5
end_topic = topic_list[end_index].split('/')[-1]

for part_url in tqdm(topic_list[start_index:end_index]):
    url = base_url + part_url
    driver.get(url)
    topic = part_url.split('/')[-1]
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    n = 0
    start = time.time()
    while n < 5:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        n += 1
        
        if n % 100 == 0:
            end = time.time()
            print(topic, n, '번', end - start)
    
    tmp_list = get_content_url(driver, topic)
    print(topic, len(tmp_list), '개')
    content_list.extend(tmp_list)
    
    driver.delete_all_cookies()

  0%|          | 0/5 [00:00<?, ?it/s]

자유여행 88 개
몽골여행 87 개
여권 94 개
패키지여행 88 개
여행지 91 개


In [31]:
content_df = pd.DataFrame(content_list,
                          columns = ['URL', '제목', '부제목', '작성자','콘텐츠', '작성일', '키워드'])
content_df

,URL,제목,부제목,작성자,콘텐츠,작성일,키워드
0,/@@fiTV/20,"로키의 꽃 ""아이스필드 파크웨이""",캐나다 기행문 7,서진이네,DAY6 - 재스퍼 to 밴프 DAY6는 재스퍼에서 밴프로 이동하는 날이다. 재스퍼...,Aug 10. 2023,자유여행
1,/@@1egW/191,프랑스 2주 자유여행(준비편),해외 자유여행 고수의 유용한 Tip,여행작가 히랑,"프랑스 2주 자유여행(준비편) ​ 간만에 동영상 공개합니다. 그동안 뉴욕, 뉴욕 했...",Aug 06. 2023,자유여행
2,/@@fkAL/45,빠니보틀처럼,그냥 가자,바카롱,나는 팩키지보다 자유여행을 선호한다. 아예 처음부터 그렇게 시작해서이기도 하고 의외...,Aug 01. 2023,자유여행
3,/@@8sbv/346,여행 예찬,자유를 향해 뛰어드는 삶,흔한여신,떠나기 전엔 미처 몰랐다. 얼마 전까지만 해도 나는 여행에 돈 쓰는 게 제일 ...,Jul 31. 2023,자유여행
4,/@@eySH/64,제주도 중독자,산굼부리의 꾸냥,임히엔,나의 첫 제주는 당시 근무하던 회사의 친한 동료분들과 함께였다. 제주가 처음이었던 ...,Jul 28. 2023,자유여행
...,...,...,...,...,...,...,...
443,/@@1jPF/1366,"대만 타이베이 가볼만한곳, 꼭 가봐야 할 여행지 7곳","대만 여행지 추천, 대만 타이베이 여행, 타이페이 가볼만한곳",호텔스닷컴,"대만의 수도, 타이베이는 경제, 문화적으로 중심적인 역할을 하는 도시입니다. 사실 ...",Jan 16. 2023,여행지
444,/@@6QjG/306,아침형 인간이 여행지에서의 아침 보내는 방법에 대하여,,순간 수집가,나이가 들면서 아침에 일찍 일어나게 되었다. 이유는 모르겠지만 새벽 4시~5시면 알...,Jan 05. 2023,여행지
445,/@@6DA7/111,되돌아보기 그리고 시작,,lee나무,지나갔네요. 시간이. 1년이. 한 해를 마무리하는 즈음에 힘들고 성가신 일들을 해...,Jan 04. 2023,여행지
446,/@@dAVT/25,<MBTI 추천 여행지> ENTJ편,지극히 개인적인 견해로 추천하는 MBTI 별 여행지,Hopeyou,트폴레옹 ENTJ 추천 여행지: Italia_ Ravello 이탈리아 라벨로 ...,Jan 04. 2023,여행지


In [32]:
driver.quit()

In [33]:
content_df.to_csv('Brunch 콘텐츠 URL_{start_topic}-{end_topic}.csv'.format(start_topic = start_topic, end_topic = end_topic),
                  index = False)